In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from database_io.db_handler import DB_handler
import pymc as pm
import numpy as np
import pandas as pd

import plotly.express as px

In [ ]:
db = DB_handler()

In [ ]:
games = db.games.get_all_games(0.1)

In [ ]:
# games["minutes_missed"] = (90 - games["minutes"]).clip(lower=0)
games["elo_diff"] = games["elo"] - games["opposition_elo"]
games["goal_diff"] = games["result"].apply(lambda x: int(x.split("-")[0]) - int(x.split("-")[1]))
games["goal_home"] = games["result"].apply(lambda x: int(x.split("-")[0]))
games["goal_away"] = games["result"].apply(lambda x: int(x.split("-")[1]))


In [ ]:
games["elo_diff"] = games["elo_diff"] / (elo_std:=max(games["elo_diff"].max(), abs(games["elo_diff"].min())))
# games["goal_diff"] = games["goal_diff"] / (goal_std:=max(games["goal_diff"].max(), abs(games["goal_diff"].min())))
games["minutes_missed"] = (games["game_minutes"] - games["minutes"]).clip(lower=0)

# games["minutes_missed"] = games["minutes_missed"] / (min_std:=games["minutes_missed"].max())

In [ ]:
games["minutes_missed"] = (games["minutes_missed"] / (minutes_missed_std:=max(games["minutes_missed"].max(), abs(games["minutes_missed"].min()))))


In [ ]:
games["minutes_missed_abs"] = games["minutes_missed"].abs()
games["goal_diff_abs"] = games["goal_diff"].abs()

In [ ]:
print(elo_std, minutes_missed_std)

In [ ]:
games

In [ ]:
import pyro
import pyro.distributions as dist
import pyro.infer as infer
import pyro.optim as optim
import torch

In [ ]:
df = games[["home", "elo_diff", "minutes_missed", "goal_diff", "goal_home", "goal_away"]]
train = torch.tensor(df.values, dtype=torch.float)
home, elo_diff, minutes_missed, goal_diff, goal_home, goal_away = train[:, 0], train[:, 1], train[:, 2], train[:, 3], train[:, 4], train[:, 5]

In [ ]:
import math

def model(is_home, elo_diff, minutes_missed, goal_home = None, goal_away = None):
    home_advantage = pyro.sample("home_advantage", dist.Normal(0.3, 1.))
    elo_diff_coeff_home = pyro.sample("elo_diff_coeff_home", dist.Normal(1., 1.))
    elo_diff_coeff_away = pyro.sample("elo_diff_coeff_away", dist.Normal(1., 1.))
    minutes_missed_coeff = pyro.sample("minutes_missed_coeff", dist.Normal(1., 1.))
    #b_ar = pyro.sample("bAR", dist.Normal(0., 1.))
    sigma = pyro.sample("sigma", dist.HalfNormal(0.1))

    mean_home = ((is_home * home_advantage) + (elo_diff * elo_diff_coeff_home)) * torch.abs(1 - (minutes_missed * minutes_missed_coeff))
    mean_away = (((-1 * elo_diff) * elo_diff_coeff_away)) * torch.abs(1 - (minutes_missed * minutes_missed_coeff))

    mean_home = torch.exp(mean_home)
    mean_away = torch.exp(mean_away)

    with pyro.plate("data", len(mean_home)):
        home = pyro.sample("home", dist.Poisson(mean_home), obs=goal_home)
#with pyro.plate("data", len(mean_away)):
        away = pyro.sample("away", dist.Poisson(mean_away), obs=goal_away)

    # with pyro.plate("data", len(home)):
    #     return pyro.sample("obs", dist.Delta(home - away))
        return pyro.sample("obs", dist.Delta(home - away))

pyro.render_model(model, model_args=(home,
                                     elo_diff, 
                                     minutes_missed, 
                                     goal_home, 
                                     goal_away), render_distributions=True)

In [ ]:
auto_guide = pyro.infer.autoguide.AutoNormal(model)
adam = pyro.optim.Adam({"lr": 0.02})
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%%time
pyro.clear_param_store()

# These should be reset each training loop.
auto_guide = pyro.infer.autoguide.AutoNormal(model)
adam = pyro.optim.Adam({"lr": 0.02})  # Consider decreasing learning rate.
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)

losses = []
for step in range(1000):  # Consider running for more steps.
    loss = svi.step(home, elo_diff, minutes_missed, goal_home, goal_away)
    losses.append(loss)
    if step % 100 == 0:
        print("Elbo loss: {}".format(loss))

plt.figure(figsize=(5, 2))
plt.plot(losses)
plt.xlabel("SVI step")
plt.ylabel("ELBO loss");

In [ ]:
predictive = pyro.infer.Predictive(model, guide=auto_guide, num_samples=800)
svi_samples = predictive(home, elo_diff, minutes_missed,  goal_diff=None)
svi_goal_diff = svi_samples["obs"]

In [ ]:
predictions = pd.DataFrame({
    "home": home,
    "elo_diff": elo_diff,
    "minutes_missed": minutes_missed, 
    "y_mean": svi_goal_diff.mean(0).detach().cpu().numpy(),
    "y_perc_5": svi_goal_diff.kthvalue(int(len(svi_goal_diff) * 0.2), dim=0)[0].detach().cpu().numpy(),
    "y_perc_95": svi_goal_diff.kthvalue(int(len(svi_goal_diff) * 0.8), dim=0)[0].detach().cpu().numpy(),
    "goal_diff": goal_diff,
})

predictions = predictions.sample(100)

home_teams = predictions[predictions["home"] == 1].sort_values(by=["elo_diff"])
away_teams = predictions[predictions["home"] == 0].sort_values(by=["elo_diff"])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), sharey=True)
fig.suptitle("Posterior predictive distribution with 80% CI", fontsize=16)

ax[0].plot(away_teams["elo_diff"], away_teams["y_mean"])
ax[0].fill_between(away_teams["elo_diff"], away_teams["y_perc_5"], away_teams["y_perc_95"], alpha=0.5)
ax[0].plot(away_teams["elo_diff"], away_teams["goal_diff"], "o")
ax[0].set(xlabel="Elo Diff", ylabel="Goal Diff", title="Away")

ax[1].plot(home_teams["elo_diff"], home_teams["y_mean"])
ax[1].fill_between(home_teams["elo_diff"], home_teams["y_perc_5"], home_teams["y_perc_95"], alpha=0.5)
ax[1].plot(home_teams["elo_diff"], home_teams["goal_diff"], "o")
ax[1].set(xlabel="Elo Diff", ylabel="Goal Diff", title="Home");

In [ ]:
# (is_home * home_advantage) + (elo_diff * elo_diff_coeff) - (minutes_missed * minutes_missed_coeff))
home_test = 0
elo_diff_test = 70 / elo_std
minutes_missed_test = 45 / minutes_missed_std

(home_test * trace.posterior.home_advantage.values[0][0] + elo_diff_test * trace.posterior.elo_diff_coeff.values[0][0]) * abs(1 -  minutes_missed_test * trace.posterior.minutes_missed_coeff.values[0][0])

In [ ]:
trace.to_netcdf("/home/morten/Develop/packing-report/metrics/mov_elo/mov_elo.nc")